In [1]:
import sys
import importlib
import datetime
import os

import numpy as np
import pandas as pd
import xarray as xr
import pvlib as pv
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.colorbar as colorbar
import scipy.ndimage.filters as filters
from distributed import LocalCluster, Client

In [2]:
%matplotlib inline

In [ ]:
%matplotlib notebook

In [3]:
sys.path.append('/home/travis/python_code/letkf_forecasting/')
import letkf_forecasting as lf
import prepare_sat_data as prep
import random_functions as rf

In [4]:
sensor_data = pd.read_hdf('/home/travis/python_code/letkf_forecasting/data/sensor_data.h5')
sensor_CI = pd.read_hdf('/home/travis/python_code/letkf_forecasting/data/sensor_CI.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/letkf_forecasting/data/sensor_loc.h5')

In [5]:
sensor_loc.sort_values(by='id', inplace=True)

In [6]:
n_workers = 20
cluster = LocalCluster(n_workers=n_workers, scheduler_port=7001, diagnostics_port=7002)
client = Client(cluster)

In [21]:
month = 5
day = 29
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [22]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [23]:
U.index

DatetimeIndex(['2014-05-29 05:00:00-07:00', '2014-05-29 06:00:00-07:00',
               '2014-05-29 07:00:00-07:00', '2014-05-29 08:00:00-07:00',
               '2014-05-29 09:00:00-07:00'],
              dtype='datetime64[ns, MST]', freq=None)

In [9]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
# crop fields

In [10]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [11]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [12]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
# side_length = 800 # 200 km
# we_width = west_east_max - west_east_min
# if we_width%2 != 0:
#     west_east_min -= 1
#     we_width = west_east_max - west_east_min
# we_border = (side_length - we_width)/2
# left = west_east_min - we_border
# right = west_east_max + we_border + 1
# west_east_range = np.arange(left, right, dtype='int')
# west_east_range_stag = np.arange(left, right + 1, dtype='int')


# sn_width = south_north_max - south_north_min
# if sn_width%2 != 0:
#     south_north_min -= 1
#     sn_width = south_north_max - south_north_min
# sn_border = (side_length - sn_width)/2
# down = int(round(south_north_min - sn_border))
# up = int(round(south_north_max + sn_border + 1))
# south_north_range = np.arange(down, up, dtype='int')
# south_north_range_stag = np.arange(down, up + 1, dtype='int')

# crop_shape = (south_north_range.size, west_east_range.size)
# U_crop_shape = (south_north_range.size, west_east_range_stag.size)
# V_crop_shape = (south_north_range_stag.size, west_east_range.size)
# print('X length: ' + 
#       str(west_east_range.size))
# print('Y length: ' + 
#       str(south_north_range.size))

In [13]:
U.values.mean()

-4.6355400802968383

In [14]:
V.values.mean()

12.346545153481648

In [15]:
max_forecast = 45
if U.values.mean()>0:
    U_max = U.values.max() # know U is positive
    left = int(U_max*60*max_forecast/250) + 10
    right = 10
else:
    U_min = U.values.min() # know U is negative
    left = 10
    right = int(abs(U_min)*60*max_forecast/250) + 10

if V.values.mean()>0:  
    V_max = V.values.max() # know V is positive
    down = int(V_max*60*max_forecast/250) + 10
    up = 10
else:
    V_min = V.values.min()
    down = 10
    up = int(abs(V_min)*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

X length: 358
Y length: 551


In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 5 # 2.5 # 10 # 1
Ly = 5 # 2.5 # 10 # 1
tol = 0.005
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
np.save('./data/random_fun/e', e)
np.save('./data/random_fun/v', v)
np.save('./data/random_fun/approx_var', approx_var)

In [ ]:
e = np.load('./data/random_fun/e.npy')
v = np.load('./data/random_fun/v.npy')
approx_var = np.load('./data/random_fun/approx_var.npy')

In [ ]:
import time

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat
time0 = time.time()
## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
# start_time = None
# end_time = None
start_time = '2014-05-29 14:00:00' #11:00:00 is not a bad start
end_time = '2014-05-29 15:00:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

time1 = time.time()
print('It took: ' + str((time1 - time0)/60))

In [ ]:
print('It took: ' + str((time1 - time0)/60))

In [ ]:
plt.figure()
plt.pcolormesh(advected_15.iloc[1].values[flat_error_domain].reshape(error_domain_shape), cmap='Blues')

plt.figure()
plt.pcolormesh(advected_30.iloc[1].values[flat_error_domain].reshape(error_domain_shape), cmap='Blues')

plt.figure()
plt.pcolormesh(advected_45.iloc[1].values[flat_error_domain].reshape(error_domain_shape), cmap='Blues')

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)

# THIS IS THE START 5 29 pert

In [ ]:
month = 5
day = 29
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [ ]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [ ]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
max_forecast = 45
U_max = U.values.max() # know U is positive
V_max = V.values.max() # know V is positive

left = int(U_max*60*max_forecast/250) + 10
right = 10

up = 10
down = int(V_max*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 2.5 # 10 # 1
Ly = 2.5 # 10 # 1
tol = 0.001
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat

## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
start_time = None
end_time = None
# start_time = '2014-04-18 12:30:00' #11:00:00 is not a bad start
# end_time = '2014-04-18 13:30:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)

# THIS IS THE START 4 18 pert

In [ ]:
month = 4
day = 18
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [ ]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [ ]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
max_forecast = 45
U_max = U.values.max() # know U is positive
V_max = V.values.max() # know V is positive

left = int(U_max*60*max_forecast/250) + 10
right = 10

up = 10
down = int(V_max*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 2.5 # 10 # 1
Ly = 2.5 # 10 # 1
tol = 0.001
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat

## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
start_time = None
end_time = None
# start_time = '2014-04-18 12:30:00' #11:00:00 is not a bad start
# end_time = '2014-04-18 13:30:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)

# THIS IS THE START 6 11 pert

In [ ]:
month = 6
day = 11
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [ ]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [ ]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
max_forecast = 45
U_max = U.values.max() # know U is positive
V_max = V.values.max() # know V is positive

left = int(U_max*60*max_forecast/250) + 10
right = 10

up = 10
down = int(V_max*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 2.5 # 10 # 1
Ly = 2.5 # 10 # 1
tol = 0.001
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat

## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
start_time = None
end_time = None
# start_time = '2014-04-18 12:30:00' #11:00:00 is not a bad start
# end_time = '2014-04-18 13:30:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)

# THIS IS THE START 4 9 pert

In [ ]:
month = 4
day = 9
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [ ]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [ ]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
max_forecast = 45
U_max = U.values.max() # know U is positive
V_max = V.values.max() # know V is positive

left = int(U_max*60*max_forecast/250) + 10
right = 10

up = 10
down = int(V_max*60*max_forecast/250) + 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 2.5 # 10 # 1
Ly = 2.5 # 10 # 1
tol = 0.001
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat

## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
start_time = None
end_time = None
# start_time = '2014-04-18 12:30:00' #11:00:00 is not a bad start
# end_time = '2014-04-18 13:30:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)

# THIS IS THE START 5 9 pert

In [ ]:
month = 5
day = 9
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + '{var}'

In [ ]:
csi = pd.read_hdf(file.format(var='csi.h5'))
x_sat = np.load(file.format(var='x.npy'))
y_sat = np.load(file.format(var='y.npy'))
domain_shape = np.load(file.format(var='domain_shape.npy'))
U = pd.read_hdf(file.format(var='U.h5'))
V = pd.read_hdf(file.format(var='V.h5'))
U_shape = np.load(file.format(var='U_shape.npy'))
V_shape = np.load(file.format(var='V_shape.npy'))

In [ ]:
## filter winds
filter_len = 30*4
U_smooth = U.values.reshape(
    [U.shape[0], U_shape[0], U_shape[1]])
U_smooth = filters.uniform_filter(
    U_smooth, size=(0, filter_len, filter_len))
this_shape = U_smooth.shape
U_smooth = U_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
U_smooth = pd.DataFrame(data=U_smooth, index=U.index)

V_smooth = V.values.reshape(
    [V.shape[0], V_shape[0], V_shape[1]])
V_smooth = filters.uniform_filter(
    V_smooth, size=(0, filter_len, filter_len))
this_shape = V_smooth.shape
V_smooth = V_smooth.reshape(this_shape[0], this_shape[1]*this_shape[2])
V_smooth = pd.DataFrame(data=V_smooth, index=V.index)

In [ ]:
x_range = x_sat
y_range = y_sat
x_sat, y_sat = np.meshgrid(x_sat, y_sat)
x_sat = x_sat.ravel()
y_sat = y_sat.ravel()

In [ ]:
lon_sens = sensor_loc['lon'].values
lat_sens = sensor_loc['lat'].values

In [ ]:
x_sens, y_sens = prep.sphere_to_lcc(lat_sens, lon_sens)
x_min = np.floor(x_sens.min())
x_max = np.ceil(x_sens.max())
y_min = np.floor(y_sens.min())
y_max = np.ceil(y_sens.max())
west_east_min = np.argmin(abs(x_min - x_range))
west_east_max = np.argmin(abs(x_max - x_range))
south_north_min = np.argmin(abs(y_min - y_range))
south_north_max = np.argmin(abs(y_max - y_range))

In [ ]:
U.values.mean()

In [ ]:
V.values.mean()

In [ ]:
max_forecast = 45
U_max = U.values.max() # know U is positive
V_max = -V.values.min() # know V is negative

left = int(U_max*60*max_forecast/250) + 10
right = 10

up = int(V_max*60*max_forecast/250) + 10
down = 10

west_east_range = np.arange(west_east_min - left, west_east_max + right + 1, dtype='int')
west_east_range_stag = np.arange(west_east_min - left, west_east_max + right + 1 + 1, dtype='int')
south_north_range = np.arange(south_north_min - down, south_north_max + up + 1, dtype='int')
south_north_range_stag = np.arange(south_north_min - down, south_north_max + up + 1 + 1, dtype='int')
crop_shape = (south_north_range.size, west_east_range.size)
U_crop_shape = (south_north_range.size, west_east_range_stag.size)
V_crop_shape = (south_north_range_stag.size, west_east_range.size)
print('X length: ' + 
      str(west_east_range.size))
print('Y length: ' + 
      str(south_north_range.size))

In [ ]:
west_east_crop, south_north_crop = np.meshgrid(west_east_range, south_north_range)
crop_indicies_2d = np.array([south_north_crop.ravel(),
                             west_east_crop.ravel()])
flat_crop_indicies = np.ravel_multi_index(crop_indicies_2d,
                                          domain_shape)

In [ ]:
west_east_crop_U, south_north_crop_U = np.meshgrid(west_east_range_stag, south_north_range)
crop_indicies_2d_U = np.array([south_north_crop_U.ravel(),
                               west_east_crop_U.ravel()])
flat_crop_indicies_U = np.ravel_multi_index(crop_indicies_2d_U,
                                            U_shape)

In [ ]:
west_east_crop_V, south_north_crop_V = np.meshgrid(west_east_range, south_north_range_stag)
crop_indicies_2d_V = np.array([south_north_crop_V.ravel(),
                               west_east_crop_V.ravel()])
flat_crop_indicies_V = np.ravel_multi_index(crop_indicies_2d_V,
                                            V_shape)

In [ ]:
x_crop = x_sat[flat_crop_indicies]
y_crop = y_sat[flat_crop_indicies]

In [ ]:
x_crop_range = x_crop.reshape(crop_shape)[0, :]
y_crop_range = y_crop.reshape(crop_shape)[:, 0]

In [ ]:
west_east_min = np.argmin(abs(x_min - x_crop_range))
west_east_max = np.argmin(abs(x_max - x_crop_range))
south_north_min = np.argmin(abs(y_min - y_crop_range))
south_north_max = np.argmin(abs(y_max - y_crop_range))
we_error_range = np.arange(west_east_min - 8, west_east_max + 9, dtype='int')
sn_error_range = np.arange(south_north_min - 8, south_north_max + 9, dtype='int')
error_domain_shape = (sn_error_range.size, we_error_range.size)
print('X length: ' + 
      str(we_error_range.size))
print('Y length: ' + 
      str(sn_error_range.size))

In [ ]:
west_east_error_crop, south_north_error_crop = np.meshgrid(
    we_error_range, sn_error_range)
crop_indicies_2d_error = np.array([south_north_error_crop.ravel(),
                             west_east_error_crop.ravel()])
flat_error_domain = np.ravel_multi_index(crop_indicies_2d_error,
                                         crop_shape)

In [ ]:
csi_crop = csi[flat_crop_indicies]
csi_crop.columns = np.arange(flat_crop_indicies.size, dtype='int')

In [ ]:
U_crop = U_smooth[flat_crop_indicies_U]
U_crop.columns = np.arange(flat_crop_indicies_U.size, dtype='int')
V_crop = V_smooth[flat_crop_indicies_V]
V_crop.columns = np.arange(flat_crop_indicies_V.size, dtype='int')

In [ ]:
importlib.reload(rf)
Lx = 2.5 # 10 # 1
Ly = 2.5 # 10 # 1
tol = 0.001
e, v = rf.eig_2d_covariance(
    x=x_crop_range, y=y_crop_range,
    Lx=Lx, Ly=Ly, tol=tol)

In [ ]:
approx_var = (v*e[None, :]*v).sum(-1).mean()

In [ ]:
#%%timeit -n1 -r1
## Assimilation only of sat to sat

## Try to use a seperate sigma for wind assimilation
importlib.reload(lf)
dx = 250 #in km
dy = 250 #in km
C_max = 0.7
assimilation_grid_size = 5 #was 1
localization_letkf = 2*4
sat_inflation = 1.5
wind_inflation = 4
wind_sat_sig = 1

goes15_azimuth = 220.5
goes15_elevation = 44.1

sat_sig = 0.05 #0.01 <- sd<rmse
ens_size = 38 #was 40
wind_sigma = (1.8, 1.8) #(1.5, 1.5)
wind_size = 2
CI_sigma = .2

pert_sigma = 0.12/3 # 0.10/3
pert_mean = 0 # -0.02/3 # 0, 0.05, 0.01
edge_weight = 1

# works: const - 3 (2, 2) .2, 0.01, loc: 1
start_time = None
end_time = None
# start_time = '2014-04-18 12:30:00' #11:00:00 is not a bad start
# end_time = '2014-04-18 13:30:00' #Gets boring shortly after 14:00:00

# U_corrected = U_const # - 3
# V_corrected = V_const

U_corrected = U_crop
V_corrected = V_crop

returned = lf.main_only_sat(
    sat=csi_crop, x=x_crop, y=y_crop, domain_shape=crop_shape,                                                                                                                                                    
    U=U_corrected, U_shape=U_crop_shape, V=V_corrected, V_shape=V_crop_shape,                                                                                                                                                            
    start_time=start_time, end_time=end_time, dx=dx, dy=dy, C_max=C_max,                                                                                                                                               
    assimilation_grid_size=assimilation_grid_size,                                                                                                                                                            
    localization_letkf=localization_letkf, sat_inflation=sat_inflation,                                                                                                                                                
    sat_sig=sat_sig, ens_size=ens_size,                                                                                                                                                                 
    wind_sigma=wind_sigma, wind_size=wind_size,
    CI_sigma=CI_sigma,
    client=client, flat_error_domain=flat_error_domain,
    wind_inflation=wind_inflation,
    wind_sat_sig=wind_sat_sig,
    pert_sigma=pert_sigma, pert_mean=pert_mean,
    rf_eig=e, rf_vectors=v, rf_approx_var=approx_var,
    edge_weight=edge_weight)
ensemble_15, ensemble_30, ensemble_45 = returned[0:3]
ensemble_analy, ens_shape = returned[3:5]
advected_15, advected_30, advected_45 = returned[5:8]

In [ ]:
suffix = '_{month}_{day}'.format(month=month, day=day)
file = './data/for' + suffix + '/' + 'results_pert/'
if not os.path.exists(file):
    os.mkdir(file)
file = file + '{var}'

In [ ]:
ensemble_15.to_hdf(file.format(var='ensemble_15.h5'), 'ensemble_15')
ensemble_30.to_hdf(file.format(var='ensemble_30.h5'), 'ensemble_30')
ensemble_45.to_hdf(file.format(var='ensemble_45.h5'), 'ensemble_45')
ensemble_analy.to_hdf(file.format(var='ensemble_analy.h5'), 'ensemble_analy')
advected_15.to_hdf(file.format(var='advected_15.h5'), 'advected_15')
advected_30.to_hdf(file.format(var='advected_30.h5'), 'advected_30')
advected_45.to_hdf(file.format(var='advected_45.h5'), 'advected_45')
np.save(file.format(var='ens_shape'), ens_shape)
np.save(file.format(var='flat_crop_indicies'), flat_crop_indicies)